In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df = pd.read_csv('covid_toy.csv')

In [7]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [25]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

## Without Column Transformer ->

In [33]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

In [35]:
oe = OrdinalEncoder()
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [41]:
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])

In [61]:
X_train_age = X_train[['age']].values
X_test_age = X_test[['age']].values

In [63]:
X_train_age.shape

(80, 1)

In [69]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_cough, X_train_gender_city), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_cough, X_test_gender_city), axis=1)

In [77]:
X_train_transformed

array([[ 80.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 80.        , 100.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 11.        , 100.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 83.        ,  98.        ,   0.        ,   1.        ,
          1.        ,   0.        ,   0.        ],
       [ 10.        , 100.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 40.        ,  98.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 22.        ,  99.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 50.        , 103.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 33.        , 102.        ,   1.        ,   0.        ,
          1.    

## With Column Transformer

In [82]:
from sklearn.compose import ColumnTransformer

In [118]:
ct = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']), 
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])
     ], remainder='passthrough')

In [128]:
ct.fit_transform(X_train).shape

(80, 7)

In [126]:
ct.transform(X_test).shape

(20, 7)